# Mallow’s Cp 

Mallow's cp is a variable selection technique based on the full model and reduced model comparison. The assumption here is the full model is unbiased; predictors must be in “correct form” and important interactions included.

As a rule of thumb, if Cp = p, then  the reduced model predicts as well as the full model, and if Cp < p, the reduced model is predicting better than the full model. In practice, we just find the model with the smallest Cp. 

In [147]:
lung_data = '''
49.0 45.0 36.0 45.0 55.0 30.0 28.0 40.0 85.0 11.0 16.0 42.0
32.0 30.0 46.0 40.0 26.0 39.0 76.0 43.0 28.0 42.0 78.0 27.0
95.0 17.0 24.0 36.0 26.0 63.0 80.0 42.0 74.0 25.0 12.0 52.0
37.0 32.0 27.0 35.0 31.0 37.0 37.0 55.0 49.0 29.0 34.0 47.0
38.0 26.0 32.0 28.0 41.0 38.0 45.0 30.0 12.0 38.0 99.0 26.0
44.0 25.0 38.0 47.0 29.0 27.0 51.0 44.0 40.0 37.0 32.0 54.0
31.0 34.0 40.0 36.0
'''
import numpy as np 
data = lung_data.splitlines()
rec = []
for line in data:
    if line:
        line_ = [float(l) for l in line.split(' ')]
        rec.extend(line_)
        
lung_df = np.array(rec).reshape(19,4)
lung_df = pd.DataFrame(lung_df)
lung_df.columns = ['y', 'x1', 'x2', 'x3']

lung_df['x1sq'] = lung_df['x1']**2
lung_df['x2sq'] = lung_df['x2']**2
lung_df['x3sq'] = lung_df['x3']**2
lung_df['x1x2'] = lung_df['x1']*lung_df['x2']
lung_df['x1x3'] = lung_df['x1']*lung_df['x3']
lung_df['x2x3'] = lung_df['x2']*lung_df['x3']

In [150]:
from itertools import combinations, chain
import statsmodels.api as sm
import statsmodels.formula.api as smf

var_list = lung_df.columns.tolist()
var_list.remove('y')

full_model = 'y~' + '+'.join(var_list)
benckmark_for_cp = smf.ols(full_model, data=lung_df).fit()
denom = benckmark_for_cp.mse_resid

make_job_list = [ list(combinations(var_list, i+1)) for i in range(len(var_list))]
make_job_list = (chain(*make_job_list))
make_job_list  = list(make_job_list)

model_fit_rec = []
for variables in make_job_list:
    formula_ = '~'.join(['y', '+'.join(variables)])
    cov_count = len(variables) + 1
    m = smf.ols(formula_, data=lung_df).fit()
    sse = m.mse_resid*(len(lung_df) - cov_count)
    metric = sse/denom - len(lung_df) + 2*cov_count

    model_fit_rec.append([metric, formula_])
model_fit_rec.sort(reverse=False)
print(model_fit_rec[0:5]) #best 5 models

[[-0.05614368976066331, 'y~x1+x2+x1x2'], [0.6717263113549006, 'y~x1+x2+x1sq'], [1.2139792287626836, 'y~x1+x2+x1sq+x2sq'], [1.3025266870131702, 'y~x1+x3+x1sq+x2x3'], [1.4107917687726346, 'y~x1+x1sq+x1x3+x2x3']]
